In [3]:
# pip install cchardet
!pip install fake-useragent

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, urldefrag
import time
import re
import collections # For deque
from fake_useragent import UserAgent

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36


In [ ]:
print(hello)

In [ ]:
# --- Configuration ---
ua = UserAgent()
REQUEST_TIMEOUT = 10  
CRAWL_DELAY = 1       
MAX_PAGES_PER_DOMAIN = 100 


--- Starting crawl for domain: nlv.gov.vn (User-Agent: Chrome-like) ---
  [1/100] Scraping: https://nlv.gov.vn
  [2/100] Scraping: https://nlv.gov.vn/dich-vu-thu-vien/thong-tin-may-chu-z39.50-cua-thu-vien-quoc-gia-viet-nam.html
  [3/100] Scraping: https://nlv.gov.vn/tin-tuc-su-kien/tap-huan-chuan-hoa-du-lieu-thu-muc-yeu-cau-va-phuong-phap.html
  [4/100] Scraping: https://nlv.gov.vn/dich-vu-thu-vien/dich-vu-phat-trien-phan-mem.html
  [5/100] Scraping: https://nlv.gov.vn/tai-lieu-lien-hop-quoc
  [6/100] Scraping: https://nlv.gov.vn/tin-tuc-su-kien/hoi-nghi-nghien-cuu-hoc-tap-quan-triet-nghi-quyet-hoi-nghi-trung-uong-5-khoa-xiii.html
  [7/100] Scraping: https://nlv.gov.vn/thu-muc-quoc-gia-nam
  [8/100] Scraping: https://nlv.gov.vn/ef/fr
  [9/100] Scraping: https://nlv.gov.vn/tin-tuc-su-kien
  [10/100] Scraping: https://nlv.gov.vn/phan-3-hcm-sang-mai-ten-nguoi
  [11/100] Scraping: https://nlv.gov.vn/thu-muc-quoc-gia-thang/thu-muc-quoc-gia-thang-2-2025.html
  [12/100] Scraping: https://nlv

In [ ]:
# --- Helper Functions ---

def is_valid_url(url):
    """Basic check if a URL seems valid and is HTTP/HTTPS"""
    parsed = urlparse(url)
    return bool(parsed.scheme) and parsed.scheme in ['http', 'https'] and bool(parsed.netloc)

def normalize_url(url):
    """Normalize URL: remove fragment, ensure scheme"""
    url, _ = urldefrag(url) # Remove fragment (e.g. #section)
    parsed = urlparse(url)
    if not parsed.scheme: # if url is like 'example.com/page'
        # We need a base to know if it's http or https,
        # this will be handled by urljoin later or the initial domain list
        # For now, just strip trailing slash if it's a full URL
        if parsed.netloc:
            return url.strip('/')
        return url # Return as is if it's a relative path for now
    return url.strip('/') # Remove trailing slash for consistency

def get_domain_netloc(url):
    """Extracts the network location (domain) from a URL."""
    return urlparse(url).netloc

def fetch_page(url, headers):
    """Fetches HTML content of a page."""
    try:
        response = requests.get(url, headers=headers, timeout=REQUEST_TIMEOUT)
        response.raise_for_status()  # Raises an HTTPError for bad responses (4XX or 5XX)
        if 'text/html' in response.headers.get('Content-Type', '').lower():
            return response.text
        else:
            print(f"    [Fetch] Skipped non-HTML content at {url}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"    [Fetch] Error fetching {url}: {e}")
        return None
    except Exception as e:
        print(f"    [Fetch] An unexpected error occurred for {url}: {e}")
        return None

def extract_text_content(html_content):
    """Extracts meaningful text from HTML."""
    if not html_content:
        return ""
    soup = BeautifulSoup(html_content, 'html.parser')

    # Remove script and style elements, and common non-content semantic tags
    for element in soup(["script", "style", "header", "footer", "nav", "aside", "form", "button", "noscript"]):
        element.decompose()

    # Get text and clean it up
    text = soup.get_text(separator=' ', strip=True)
    text = re.sub(r'\s+', ' ', text) # Replace multiple whitespaces with a single space
    return text.strip()

def extract_internal_links(html_content, base_url, domain_netloc):
    """Extracts all internal links from HTML content."""
    links = set()
    if not html_content:
        return links

    soup = BeautifulSoup(html_content, 'html.parser')
    for a_tag in soup.find_all('a', href=True):
        href = a_tag['href']
        # Resolve relative URLs: urljoin needs base_url to have a scheme
        parsed_base = urlparse(base_url)
        if not parsed_base.scheme:
            # This shouldn't happen if initial URLs are well-formed,
            # but as a fallback, assume http for the base if scheme is missing for urljoin
            temp_base = 'http://' + base_url if not base_url.startswith(('http://', 'https://')) else base_url
            full_url = urljoin(temp_base, href)
        else:
            full_url = urljoin(base_url, href)

        full_url = normalize_url(full_url)

        # Ensure it's a valid HTTP/HTTPS URL and belongs to the same domain
        if is_valid_url(full_url) and get_domain_netloc(full_url) == domain_netloc:
            links.add(full_url)
    return links

In [ ]:
# --- Main Crawler ---

def crawl_domain(start_url_str):
    """Crawls a single domain starting from start_url."""
    # Ensure the start_url has a scheme for proper processing
    parsed_start_url = urlparse(start_url_str)
    if not parsed_start_url.scheme:
        start_url = "http://" + start_url_str # Default to http, or use https if you know
    else:
        start_url = start_url_str
    start_url = normalize_url(start_url)
    
    domain_netloc = get_domain_netloc(start_url)
    if not domain_netloc:
        print(f"Invalid start URL (no domain): {start_url_str}. Skipping.")
        return []
        
    print(f"\n--- Starting crawl for domain: {domain_netloc} (User-Agent: Chrome-like) ---")

    queue = collections.deque()
    visited_urls = set()
    scraped_pages_content = [] # List to store {'url': url, 'content': text}

    queue.append(start_url)

    headers = {'User-Agent': ua.random}
    pages_crawled_count = 0

    while queue and pages_crawled_count < MAX_PAGES_PER_DOMAIN:
        current_url = queue.popleft()

        if current_url in visited_urls:
            continue

        visited_urls.add(current_url)
        pages_crawled_count += 1

        print(f"  [{pages_crawled_count}/{MAX_PAGES_PER_DOMAIN}] Scraping: {current_url}")

        # Politely wait (still good practice even on your own servers to avoid overload)
        time.sleep(CRAWL_DELAY)

        html_content = fetch_page(current_url, headers)
        if html_content:
            text_content = extract_text_content(html_content)
            if text_content: # Only add if there's meaningful content
                scraped_pages_content.append({
                    'url': current_url,
                    'content': text_content,
                    'domain': domain_netloc
                })

            # Extract new links to crawl
            internal_links = extract_internal_links(html_content, current_url, domain_netloc)
            for link in internal_links:
                if link not in visited_urls:
                    queue.append(link)

    print(f"--- Finished crawl for {domain_netloc}. Scraped {len(scraped_pages_content)} pages. ---")
    return scraped_pages_content

# --- Script Execution ---
if __name__ == "__main__":
    domains_to_scrape = [
        "https://nlv.gov.vn/",        #
    ]

    all_scraped_data = []

    for domain_url_base in domains_to_scrape:
        # Ensure starting URL has a scheme for urlparse to work correctly for get_domain_netloc
        # and for urljoin to resolve relative paths correctly from the very start.
        if not urlparse(domain_url_base).scheme:
            # Default to http, or choose https if your educational sites use it predominantly
            start_url = f"http://{domain_url_base}"
            print(f"Assuming http:// for {domain_url_base}, resulting in {start_url}")
        else:
            start_url = domain_url_base

        data_from_domain = crawl_domain(start_url)
        all_scraped_data.extend(data_from_domain)
        print(f"Collected {len(data_from_domain)} pages from {get_domain_netloc(start_url)}")

    print(f"\n\n--- Total pages scraped from all domains: {len(all_scraped_data)} ---")

    if all_scraped_data:
        print("\nSample of scraped data (first 2 entries):")
        for i, item in enumerate(all_scraped_data[:2]):
            print(f"\nEntry {i+1}:")
            print(f"  URL: {item['url']}")
            print(f"  Domain: {item['domain']}")
            print(f"  Content (first 200 chars): {item['content'][:200]}...")
    else:
        print("No data was scraped.")

